<a href="https://colab.research.google.com/github/anirudh201098/Store-Item-Demand-Forecasting/blob/master/fbprophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from fbprophet import Prophet
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
data1=pd.read_csv('/content/drive/My Drive/Dataset/train.csv',parse_dates=['date'],index_col=['date'])

In [3]:
data1= data1.sort_values('date', ascending=True)

In [ ]:
#data1.reset_index(level=0, inplace=True)

In [4]:
data1.head()

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-01,7,12,26
2013-01-01,7,46,27
2013-01-01,8,12,54
2013-01-01,9,12,35


In [6]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from keras.metrics import mean_absolute_percentage_error
import math

In [7]:
def proph(store,item,X_test):
  train=data1[(data1.index.year!=2017) & (data1['store']==store) & (data1['item']==item)].drop(['store','item'],axis=1)
  train.reset_index(level=0,inplace=True)
  train.columns=['ds','y']
  prophet_basic = Prophet(yearly_seasonality=True)
  prophet_basic.fit(train)
  predictions=prophet_basic.predict(X_test)['yhat']
  return predictions

In [8]:
def metrics(store,item):
  test=data1[(data1.index.year==2017)& (data1['store']==store) & (data1['item']==item)].drop(['store','item'],axis=1)
  test.reset_index(level=0,inplace=True)
  X_test=pd.DataFrame(test['date'])
  X_test.columns=['ds']
  y_test=test['sales']
  pred=proph(store,item,X_test)
  r2=r2_score(y_test,pred)
  SS_Residual = sum((y_test-pred)**2)
  adjusted_r_squared = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
  mape=float(mean_absolute_percentage_error(y_test,pred))
  mae=mean_absolute_error(y_test,pred)
  rmse=math.sqrt(mean_squared_error(y_test,pred))
  return r2,mape,mae,rmse,SS_Residual,adjusted_r_squared

In [9]:
r=[]
ma=[]
mae1=[]
rm=[]
ss=[]
ad=[]
for i in range(0,10):
  r.append([])
  ma.append([])
  mae1.append([])
  rm.append([])
  ss.append([])
  ad.append([])
  for j in range(0,50):
    r2,mape,mae,rmse,s,adj=metrics(i+1,j+1)
    r[i].append(r2)
    ma[i].append(mape)
    mae1[i].append(mae)
    rm[i].append(rmse)
    ss[i].append(s)
    ad[i].append(adj)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [11]:
print("R2 score=",np.mean(np.mean(r,axis=1)))

R2 score= 0.6441352080235327


In [17]:
print("MAE=",(np.mean(mae1,axis=1)))


MAE= [6.47178783 8.02296157 7.32032981 7.11426762 5.78334626 5.69679921
 5.46591597 7.75782537 7.01582213 7.3040475 ]


In [18]:
print("MAPE=",(np.mean(ma,axis=1)))

MAPE= [14.89331212 12.82240704 13.15421289 14.05975712 16.22911908 15.71837983
 16.76091972 13.04702772 13.72250677 13.42769177]


In [14]:
print("RMSE=",np.mean(np.mean(rm,axis=1)))

RMSE= 8.454551251014415


In [15]:
print("SSE=",np.mean(np.mean(ss,axis=1)))

SSE= 28288.959577527497


In [16]:
print("Adjusted r2 score=",np.mean(np.mean(ad,axis=1)))

Adjusted r2 score= 0.6431548642439833
